In [1]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
path=f'./data'

In [3]:
!ls {path}

labels.csv  sample_submission.csv  test  train


another way to get len <br>
len_train=!ls {path}/train | wc -l    <br>

len_train=int(str(len_train[0]))      <br>

len_train

In [4]:
len_train=len(os.listdir(f'{path}/train'))
len_test=len(os.listdir(f'{path}/test'))

In [5]:
labels=pd.read_csv(f'{path}/labels.csv')

In [6]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


Init signature: set(self, /, *args, **kwargs)  <br>
Docstring:     <br>
set() -> new empty set object <br>
set(iterable) -> new set object <br>

Build an unordered collection of unique elements. <br>
将重复项消掉

In [7]:
classes=list(set(labels['breed']))

In [8]:
n_class=len(classes)

底下的这种写法实在简洁

In [9]:
class_to_num=dict(zip(classes,range(n_class)))
num_to_class=dict(zip(range(n_class),classes))

In [10]:
class_to_num

{'affenpinscher': 38,
 'afghan_hound': 33,
 'african_hunting_dog': 110,
 'airedale': 37,
 'american_staffordshire_terrier': 2,
 'appenzeller': 60,
 'australian_terrier': 36,
 'basenji': 65,
 'basset': 27,
 'beagle': 117,
 'bedlington_terrier': 67,
 'bernese_mountain_dog': 95,
 'black-and-tan_coonhound': 71,
 'blenheim_spaniel': 13,
 'bloodhound': 100,
 'bluetick': 103,
 'border_collie': 29,
 'border_terrier': 69,
 'borzoi': 6,
 'boston_bull': 86,
 'bouvier_des_flandres': 106,
 'boxer': 108,
 'brabancon_griffon': 101,
 'briard': 15,
 'brittany_spaniel': 24,
 'bull_mastiff': 8,
 'cairn': 83,
 'cardigan': 112,
 'chesapeake_bay_retriever': 11,
 'chihuahua': 94,
 'chow': 70,
 'clumber': 4,
 'cocker_spaniel': 119,
 'collie': 18,
 'curly-coated_retriever': 1,
 'dandie_dinmont': 88,
 'dhole': 20,
 'dingo': 113,
 'doberman': 64,
 'english_foxhound': 81,
 'english_setter': 115,
 'english_springer': 39,
 'entlebucher': 92,
 'eskimo_dog': 52,
 'flat-coated_retriever': 12,
 'french_bulldog': 35,
 '

In [11]:
num_to_class

{0: 'miniature_pinscher',
 1: 'curly-coated_retriever',
 2: 'american_staffordshire_terrier',
 3: 'italian_greyhound',
 4: 'clumber',
 5: 'newfoundland',
 6: 'borzoi',
 7: 'kerry_blue_terrier',
 8: 'bull_mastiff',
 9: 'norwich_terrier',
 10: 'irish_terrier',
 11: 'chesapeake_bay_retriever',
 12: 'flat-coated_retriever',
 13: 'blenheim_spaniel',
 14: 'saint_bernard',
 15: 'briard',
 16: 'lhasa',
 17: 'mexican_hairless',
 18: 'collie',
 19: 'irish_water_spaniel',
 20: 'dhole',
 21: 'staffordshire_bullterrier',
 22: 'yorkshire_terrier',
 23: 'malamute',
 24: 'brittany_spaniel',
 25: 'scotch_terrier',
 26: 'pug',
 27: 'basset',
 28: 'leonberg',
 29: 'border_collie',
 30: 'kuvasz',
 31: 'irish_wolfhound',
 32: 'scottish_deerhound',
 33: 'afghan_hound',
 34: 'soft-coated_wheaten_terrier',
 35: 'french_bulldog',
 36: 'australian_terrier',
 37: 'airedale',
 38: 'affenpinscher',
 39: 'english_springer',
 40: 'papillon',
 41: 'komondor',
 42: 'pekinese',
 43: 'gordon_setter',
 44: 'toy_poodle',


最好是将X_train,Y_train都做成是DataFrame，这样可以直接调用to_csv<br>
但是对于图形识别的project，这是没用的。因为CV第一步都是卷积，对3层进行卷积，而DataFrame是二维的，无法卷积。除非输入图形只有一层。<br>
底下的操作就是如何将输入处理成DataFrame<br>
width=300
X_train=pd.DataFrame(np.zeros((len_train,width*width*3)),index=range(len_train))
Y_train=pd.DataFrame(np.zeros((len_train,n_class)),columns=list(class_to_num.keys()))

In [12]:
width=200
X_train=np.zeros((len_train,width,width,3))
Y_train=np.zeros((len_train,n_class))

In [13]:
fp_list=os.listdir(f'{path}/train')
n=len(fp_list)
for i in tqdm(range(n)):
    fp=fp_list[i]
    X_train[i]=cv2.resize(cv2.imread(f'{path}/train/{fp}'),dsize = (width, width))
    file_name=fp[:-4]
    breed_name=labels.loc[labels['id']==file_name]['breed']
    breed_name=breed_name.get_values()[0]
    Y_train[i][class_to_num[breed_name]]=1
    #Y_train[i][class_to_num[labels['breed'][i]]] = 1

100%|██████████| 10222/10222 [00:36<00:00, 282.91it/s]


In [14]:
print(os.listdir(f'{path}/train')[100])
print(os.listdir(f'{path}/train')[100][:-4])
print(labels.loc[labels['id']==os.listdir(f'{path}/train')[100][:-4]])
print(class_to_num['gordon_setter'])
print(Y_train[100])

66b1a3eb90d750b8739cbd6e74db13cf.jpg
66b1a3eb90d750b8739cbd6e74db13cf
                                    id          breed
4122  66b1a3eb90d750b8739cbd6e74db13cf  gordon_setter
68
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


fp_list=os.listdir(f'{path}/train')
n=len(fp_list)
for i in tqdm(range(n)):
#for fp in os.listdir(f'{path}/train'):
    fp=fp_list[i]
    X_train.iloc[i,:]=cv2.resize(cv2.imread(f'{path}/train/{fp}'),dsize = (width, width)).flatten()
    file_name=fp[:-4]
    breed_name=labels.loc[labels['id']==file_name]['breed']
    breed_name=breed_name.get_values()[0]
    Y_train.iloc[i,class_to_num[breed_name]]=1

X_train.to_csv("X_train_23_05.csv")

Y_train.to_csv("Y_train_23_05.csv")

In [15]:
#this code from kaggle which runs very fast
from keras.applications import resnet50
from keras.models import *
from keras.layers import *
#from keras.preprocessing import image
#from keras.applications.resnet50 import preprocess_input, decode_predictions
resnet_model = resnet50.ResNet50(include_top = False, weights='imagenet', input_shape = (300, 300, 3))
    
inputs = Input((width, width, 3))
x = inputs
x = resnet_model(x)
x = GlobalAveragePooling2D()(x)
resnet_model = Model(inputs, x)
resnet_model.summary()

features = resnet_model.predict(X_train, batch_size=64, verbose=1)


inputs = Input(features.shape[1:])
x = inputs
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax')(x)
model = Model(inputs, x)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

h = model.fit(features, Y_train, batch_size=128, epochs=50, validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________
10222/10222 [==============================] - 87s 9ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         

In [14]:
from keras.applications import resnet50

In [15]:
??resnet50

In [ ]:
from keras.applications import resnet50
model=Sequential()
model.add(resnet50.ResNet50(include_top = False, weights='imagenet', input_shape = (200,200,3)))
model.summary()
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class,activation='relu'))
model.summary()
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

model.fit(X_train,Y_train,batch_size=128,epochs=50,validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
___________________________________________________________

In [14]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(width,width,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class,activation='relu'))

sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

model.fit(X_train,Y_train,batch_size=128,epochs=50,validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/50
9199/9199 [==============================] - 38s 4ms/step - loss: 7.0599 - acc: 0.0064 - val_loss: 5.5462 - val_acc: 0.0059
Epoch 2/50
9199/9199 [==============================] - 29s 3ms/step - loss: 5.2640 - acc: 0.0071 - val_loss: 5.1286 - val_acc: 0.0059
Epoch 3/50
9199/9199 [==============================] - 29s 3ms/step - loss: 5.0982 - acc: 0.0073 - val_loss: 5.0882 - val_acc: 0.0059
Epoch 4/50
9199/9199 [==============================] - 30s 3ms/step - loss: 5.0289 - acc: 0.0078 - val_loss: 4.9714 - val_acc: 0.0059
Epoch 5/50
9199/9199 [==============================] - 30s 3ms/step - loss: 4.9692 - acc: 0.0076 - val_loss: 4.9527 - val_acc: 0.0059
Epoch 6/50
9199/9199 [==============================] - 29s 3ms/step - loss: 4.9534 - acc: 0.0078 - val_loss: 4.9388 - val_acc: 0.0059
Epoch 7/50
9199/9199 [==============================] - 30s 3ms/step - loss: 4.9395 - acc: 0.0074 - val_loss: 4.9267 - val_acc: 0.0059
Epoch 8

KeyboardInterrupt: 

In [ ]:
Y_test = model.predict(X_test, batch_size=128)